In [46]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3 as db

In [47]:
#Get sample data
df = pd.DataFrame(np.random.randint(0,100,size=(10000, 4)), columns=list('ABCD'))
df.head(5)

,A,B,C,D
0,82,58,6,69
1,66,93,82,32
2,19,71,3,17
3,10,44,74,68
4,45,2,46,5


In [48]:
#Create our test database for upserts (this is postgreSQL)
DB_TYPE = 'postgresql'
DB_DRIVER = 'psycopg2'
DB_USER = 'admin'
DB_PASS = 'password'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'pandas_upsert'
### Config update complete ###
SQLALCHEMY_DATABASE_URI = '%s+%s://%s:%s@%s:%s/%s' %(DB_TYPE, DB_DRIVER, DB_USER,
                                                     DB_PASS, DB_HOST, DB_PORT, DB_NAME)
#conn = db.connect('test_upsert.db')
#engine = create_engine(r'sqlite:///test_upsert.db')
engine = create_engine(SQLALCHEMY_DATABASE_URI)

In [49]:
#Create a table with a unique constraint on A and B columns.
#Insert data using pandas.to_sql
engine.execute("""DROP TABLE IF EXISTS "test_upsert" """)

engine.execute("""CREATE TABLE "test_upsert" (
                  "ID" SERIAL PRIMARY KEY,
                  "A" INTEGER,
                  "B" INTEGER,
                  "C" INTEGER,
                  "D" INTEGER) """)

df.to_sql('test_upsert', engine, if_exists='append', index=False)

In [50]:
#Check that the table exists and there is data in it
df_in_db = pd.read_sql_query('SELECT * FROM test_upsert', engine)
df_in_db.head(5)

,ID,A,B,C,D
0,1,82,58,6,69
1,2,66,93,82,32
2,3,19,71,3,17
3,4,10,44,74,68
4,5,45,2,46,5


In [51]:
#Now lets bring in some new data to insert, along with the same old data
df_new = df = pd.DataFrame(np.random.randint(0,100,size=(10000, 4)), columns=list('ABCD'))
df2 = pd.concat([df, df_new])
df2.head(1)

,A,B,C,D
0,59,38,23,57


In [52]:
#First let's get the length of both dataframe
len_df2 = df2.shape[0]
len_df_in_db = df_in_db.shape[0]
print ('new df is %s rows, and data in db is %s rows') %(len_df2, len_df_in_db)

new df is 20000 rows, and data in db is 10000 rows


In [53]:
#Now let's find out what rows are duplicates using a combination of full outer join and select where is not null
#The new _merge column added via the new indicator column in pandas will help us greatly here

df_common = pd.merge(df2, df_in_db, how='outer', on=['A', 'B'], 
                     copy=False, indicator=True, suffixes=['', '_in_db'])
df_common.reset_index(inplace=True, drop=True)


print 'outer joined df is %s rows' %(df_common.shape[0])
df_common.head(5)

outer joined df is 31071 rows


,A,B,C,D,ID,C_in_db,D_in_db,_merge
0,59.0,38.0,23.0,57.0,NaN,NaN,NaN,left_only
1,59.0,38.0,79.0,40.0,NaN,NaN,NaN,left_only
2,59.0,38.0,23.0,57.0,NaN,NaN,NaN,left_only
3,59.0,38.0,79.0,40.0,NaN,NaN,NaN,left_only
4,95.0,21.0,20.0,52.0,9587.0,26.0,62.0,both


In [56]:
#Now let's drop any columns that are in "both" or the "right only (in the datbase)
cols_to_drop = list([col for col in df_common.columns \
                     if '_in_db' in col \
                     or 'ID' in col
                     or 'index' in col
                     or '_merge' in col])
print cols_to_drop
df_unique = df_common.drop(cols_to_drop, axis=1)
df_unique.head(5)

[u'ID', u'C_in_db', u'D_in_db', '_merge']


,A,B,C,D
0,59.0,38.0,23.0,57.0
1,59.0,38.0,79.0,40.0
2,59.0,38.0,23.0,57.0
3,59.0,38.0,79.0,40.0
4,95.0,21.0,20.0,52.0


In [57]:
print 'outer joined df is %s rows' %(df_unique.shape[0])

outer joined df is 31071 rows


In [58]:
#Now let's put the new records in df_unique back into the database!
df_unique.to_sql('test_upsert', engine, if_exists='append', index=False)

In [59]:
df_in_db_2 = pd.read_sql_query('SELECT count("ID") FROM test_upsert', engine)
df_in_db_2.head(1)

,count
0,41071


In [60]:
#Awesome, that seems to work.  We only inserted new rows!  Let's check to make sure it's unique
df_dupscount = pd.read_sql_query("""
                                SELECT t."A", t."B"
                                from (select "A", "B"
                                      from test_upsert
                                      group by "A", "B"
                                      having count(1) >1 ) dups
                                      INNER JOIN test_upsert t ON
                                      (dups."A" = t."A" and dups."B" = t."B")
                                ORDER BY t."A", t."B"
                                """, engine)
df_dupscount.head(1)

,A,B
0,0,0


In [61]:
#Check if unique constraint upheld be adding to table
args = """ ALTER TABLE test_upsert ADD CONSTRAINT uk_a_b UNIQUE ("A", "B") """
engine.execute(args)
#FAILS

IntegrityError: (psycopg2.IntegrityError) could not create unique index "uk_a_b"
DETAIL:  Key ("A", "B")=(52, 35) is duplicated.
 [SQL: ' ALTER TABLE test_upsert ADD CONSTRAINT uk_a_b UNIQUE ("A", "B") ']